<div class="alert alert-block alert-info">
<b>Deadline:</b> March 22, 2023 (Wednesday) 23:00
</div>

# Exercise 2. Recommender system

In this exercise, your task is to design a recommender system.

## Learning goals:
* Practise tuning a neural network model by using different regularization methods.

In [50]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import tools
import data

In [72]:
skip_training = True  # Set this flag to True before validation and submission

In [52]:
# During evaluation, this cell sets skip_training to True
# skip_training = True

import tools, warnings
warnings.showwarning = tools.customwarn

In [53]:
# When running on your own computer, you can specify the data directory by:
# data_dir = tools.select_data_dir('/your/local/data/directory')
data_dir = tools.select_data_dir()

The data directory is /coursedata


In [54]:
# Select the device for training (use GPU if you have one)
#device = torch.device('cuda:0')
device = torch.device('cpu')

In [55]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

## Ratings dataset

We will train the recommender system on the dataset in which element consists of three values:
* `user_id` - id of the user (the smallest user id is 1)
* `item_id` - id of the movie (the smallest item id is 1)
* `rating` - rating given by the user to the item (ratings are integer numbers between 1 and 5.

The recommender system need to predict the rating for any given pair of `user_id` and `item_id`.

We measure the quality of the predicted ratings using the mean-squared error (MSE) loss:
$$
  \frac{1}{N}\sum_{i=1}^N (r_i - \hat{r}_i)^2
$$
where $r_i$ is a real rating and $\hat{r}_i$ is a predicted one.

Note: The predicted rating $\hat{r}_i$ does not have to be an integer number.

In [56]:
trainset = data.RatingsData(root=data_dir, train=True)
testset = data.RatingsData(root=data_dir, train=False)

In [57]:
# Print one sample from the dataset
x = trainset[0]
print(f'user_id={x[0]}, item_id={x[1]}, rating={x[2]}')

user_id=1, item_id=1, rating=5


# Model

You need to design a recommender system model with the API described in the cell below.

Hints on the model architecture:
* You need to use [torch.nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html?highlight=embedding#torch.nn.Embedding) layer to convert inputs `user_ids` and `item_ids` into reasonable representations. The idea of the embedding layer is that we want to represent similar users with values that are close to each other. The original representation as integers is not good for that. By using the embedding layer, we can learn such useful representations automatically.

### Model tuning

In this exercise, you need to tune the architecture of your model to achieve the best performance on the provided test set. You will notice that overfitting is a severe problem for this data: The model can easily overfit the training set producing poor accuracy on the out-of-training (test) data.

You need to find an optimal combination of the hyperparameters, with some hyperparameters corresponding to the regularization techniques that we studied in the lecture.

The hyperparameters that you are advised to consider:
* Learning rate value and learning rate schedule (decresing the learning rate often has positive effect on the model performance)
* Number of training epochs
* Network size
* Weight decay
* Early stopping
* Dropout
* Increase amount of data:
  * Data augmentation
  * Injecting noise

You can tune the hyperparameters by, for example, grid search, random search or manual tuning. In that case, you can use `architecture` argument to specify the hyperparameters that define the architecture of your network. After you have tuned the hyperparameters, set the default value of this argument to the optimal set of the hyparameters so that the best architecture is used in the accuracy tests.

Note:
* The number of points that you will get from this exercise depends on the MSE loss on the test set:
  * below 1.00: 1 point
  * below 0.95: 2 points
  * below 0.92: 3 points
  * below 0.90: 4 points
  * below 0.89: 5 points
  * below 0.88: 6 points 

In [58]:
class RecommenderSystem(nn.Module):
    def __init__(self, n_users, n_items,
                 architecture=None  # If you want to tune the hyperparameters automatically (e.g. using random
                                    # search), use this argument to specify the hyperparameters that define the
                                    # architecture of your network. After you have tuned the hyperparameters,
                                    # set the default value of this argument to the optimal set of the hyparameters
                                    # so that the best architecture is used in the accuracy tests.
                ):
        """
        Args:
          n_users: Number of users.
          n_items: Number of items.
        """
        # YOUR CODE HERE
        # raise NotImplementedError()
        # an embedding dimension refers to the number of dimensions used to represent each item in 
        # an embedding layer. For example, if you have an embedding layer for user_id and you set the 
        # embedding dimension to 100, each user_id will be represented by a vector with 100 dimensions.

        # The choice of embedding dimension is a hyperparameter that needs to be tuned based on 
        # the size of the dataset and the complexity of the problem. A smaller embedding dimension 
        # can reduce the amount of memory required by the model and make it faster to train, 
        # but it can also reduce the accuracy of the model. On the other hand, a larger embedding 
        # dimension can increase the accuracy of the model, but it can also make the model slower 
        # and more memory-intensive.

        # In general, a good strategy for choosing the embedding dimension is to start with 
        # a small value and gradually increase it until the performance of the model stops improving
        
        super(RecommenderSystem, self).__init__()
        
        user_embedding_dim = 100
        item_embedding_dim = 100
        
        self.embedUsers = nn.Embedding(num_embeddings=n_users+1, embedding_dim=user_embedding_dim)
        self.embedItems = nn.Embedding(num_embeddings=n_items+1, embedding_dim=item_embedding_dim)
        
        # Linear layers
        # Note: the input layer size is the combined embedding dimensions of user and item
        self.drop0 = nn.Dropout(0.02)
        self.fc1 = nn.Linear(user_embedding_dim + item_embedding_dim, 100)
        self.relu1 = nn.ReLU()
        self.drop1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(100, 100)
        self.relu2 = nn.ReLU()
        self.drop2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(100, 1)
        self.sigmoid = nn.Sigmoid()
        
        
    def forward(self, user_ids, item_ids):
        """
        Args:
          user_ids of shape (batch_size): User ids (starting from 1).
          item_ids of shape (batch_size): Item ids (starting from 1).
        
        Returns:
          outputs of shape (batch_size): Predictions of ratings.
        """
        # Embedding lookup
        user_embed = self.embedUsers(user_ids)
        item_embed = self.embedItems(item_ids)

        # After creating the embedding layers for users and items, 
        # we can concatenate the two embeddings into a single feature vector 
        # and pass it through a feedforward neural network.
        
        x = torch.cat([user_embed, item_embed], dim=1)
        x = self.drop0(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.drop1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.drop2(x)
        x = self.fc3(x)
        # the range of the sigmoid function is between 0 and 1
        x = self.sigmoid(x)
        # By multiplying by 5, the rating values can be between 0 and 5.  
        x = x.flatten() * 5
        return x
        
        

        
        # YOUR CODE HERE
        # raise NotImplementedError()

You can test the shapes of the model outputs using the function below.

In [59]:
def test_RecommenderSystem_shapes():
    n_users, n_items = 100, 1000
    model = RecommenderSystem(n_users, n_items)
    batch_size = 10
    user_ids = torch.arange(1, batch_size+1)
    item_ids = torch.arange(1, batch_size+1)
    output = model(user_ids, item_ids)
    print(output.shape)
    assert output.shape == torch.Size([batch_size]), "Wrong output shape."
    print('Success')

test_RecommenderSystem_shapes()

torch.Size([10])
Success


In [60]:
# This cell is reserved for testing

## Train the model

You need to train a recommender system using **only the training data.** Please use the test set to select the best model: the model that generalizes best to out-of-training data.

**IMPORTANT**:
* During testing, the predictions are produced by `predictions = model(user_ids, item_ids)` with the `user_ids` and `item_ids` loaded from `RatingsData`.
* There is a size limit of 30Mb for saved models.

In [61]:
# Create the model
# IMPORTANT: the default value of the architecture argument should define your best model.
model = RecommenderSystem(trainset.n_users, trainset.n_items)

In [62]:

# Scheduler for decreasing learning rate
from torch.optim.lr_scheduler import StepLR

# Class for the early stopping
class EarlyStopping:
    def __init__(self, tolerance, patience):
        """
        Args:
          patience (int):    Maximum number of epochs with unsuccessful updates.
          tolerance (float): We assume that the update is unsuccessful if the validation error is larger
                              than the best validation error so far plus this tolerance.
        """
        self.tolerance = tolerance
        self.patience = patience
    
    def stop_criterion(self, val_errors):
        """
        Args:
          val_errors (iterable): Validation errors after every update during training.
        
        Returns: True if training should be stopped: when the validation error is larger than the best
                  validation error obtained so far (with given tolearance) for patience epochs (number of consecutive epochs for which the criterion is satisfied).
                 
                 Otherwise, False.
        """
        if len(val_errors) <= self.patience:
            return False

        min_val_error = min(val_errors)
        val_errors = np.array(val_errors[-self.patience:])
        return all(val_errors > min_val_error + self.tolerance)
    
# This is the function to compute the loss:
def compute_loss(model, users, items, ratings):
    model.eval()
    with torch.no_grad():
        outputs = model.forward(users, items)
        loss = F.mse_loss(outputs, ratings)
        return loss.cpu().numpy()

# This is the function to print the progress during training
def print_progress(epoch, train_error, val_error):
    print('Epoch {}: Train error: {:.4f}, Test error: {:.4f}'.format(
        epoch, train_error, val_error))
    
# skip_training=False
# Implement the training loop in this cell

if not skip_training:
    # YOUR CODE HERE
    # raise NotImplementedError()

    learning_rate = 0.0001
    epochs = 50
    L2 = 0.001
    early_stop = EarlyStopping(tolerance=0.001, patience=5)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=L2)
    
    train_errors = []
    val_errors = []
    
    # The step size is the number of epochs after which to decrease the learning rate, 
    # and the gamma is the factor by which to multiply the learning rate at each step
    scheduler = StepLR(optimizer, step_size=2, gamma=0.95)
    
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

    for epoch in range(epochs):
        model.train()
        for user_ids, item_ids, ratings in trainloader:
            optimizer.zero_grad()
            ratings = ratings.to(torch.float32)
            outputs = model.forward(user_ids, item_ids)
            loss = F.mse_loss(outputs, ratings)
            loss.backward()
            optimizer.step()

        scheduler.step()
        
        trainUsers, trainItems, trainRatings = trainset[:][0], trainset[:][1], trainset[:][2]
        testUsers, testItems, testRatings = testset[:][0], testset[:][1], testset[:][2]
        train_errors.append(compute_loss(model, trainUsers, trainItems, trainRatings))
        val_errors.append(compute_loss(model, testUsers, testItems, testRatings))
        
        print_progress(epoch, train_errors[-1], val_errors[-1])
    
        if early_stop.stop_criterion(val_errors):
            print(val_errors[epoch])
            print(f"Stopping epoch: {epoch + 1}")
            break
    
    plt.plot(range(epoch+1), train_errors, label = 'training loss')
    plt.plot(range(epoch+1), val_errors, label = 'validation loss')
    plt.legend()
    print('Final test error:', val_errors[-1])

In [63]:
# Save the model to disk (the pth-files will be submitted automatically together with your notebook)
# Set confirm=False if you do not want to be asked for confirmation before saving.
if not skip_training:
    tools.save_model(model, 'recsys.pth', confirm=False)

In [64]:
# This cell loads your best model
if skip_training:
    model = RecommenderSystem(trainset.n_users, trainset.n_items)
    tools.load_model(model, 'recsys.pth', device)

Model loaded from recsys.pth.


The next cell tests the accuracy of your best model. It is enough to submit .pth files.

**IMPORTANT**:
* During testing, the predictions are produced by `predictions = model(user_ids, item_ids)` with the `user_ids` and `item_ids` loaded from `RatingsData`.
* There is a size limit of 30Mb for saved models. Please make sure that your model loads in the cell above.

In [65]:
# This cell tests the accuracy of your best model.

In [66]:
# This cell is reserved for grading

In [67]:
# This cell is reserved for grading

In [68]:
# This cell is reserved for grading

In [69]:
# This cell is reserved for grading

In [70]:
# This cell is reserved for grading

In [71]:
# This cell is reserved for grading